This notebook is for practicing how to use make and train convolutional neural network using tenesorflow. The model is able to achieve 98.5% accuracy on MNIST dataset after 8,000 iterations. The accuracy can be improved by tra

I give full credit to Magnus Erik Hvass Pedersen as a lot of the work here is influenced by him. You can find him at  http://www.hvass-labs.org/

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

C:\Users\tep\Anaconda3\envs\untitled1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
data_row, data_col = 28, 28
data_shape = (data_row, data_col)
data_size = data_row * data_col
class_size = 10
num_channels = 1

A helper funtion that creates a convolutional layer.

In [4]:
def createConvLayer(input,              
                   num_input_channels, 
                   filter_size,        
                   num_filters):
    
    shape = [filter_size, filter_size, num_input_channels, num_filters]
    
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    
    biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))
    
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=[1, 1, 1, 1],
                         padding='SAME')
    
    layer = tf.nn.max_pool(value=layer,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME')
    layer = layer + biases
    
    layer = tf.nn.relu(layer)
    
    return layer, weights

In [5]:
x = tf.placeholder(tf.float32, shape=[None, data_size], name='x')

In [6]:
x_image = tf.reshape(x, [-1, data_row, data_col, num_channels])

In [7]:
y_true = tf.placeholder(tf.float32, shape=[None, class_size], name='y_true')
y_true_cls = tf.argmax(y_true, axis=1)

In [8]:
filter_size1 = 5          
num_filters1 = 16         

filter_size2 = 5          
num_filters2 = 36    

fc_size = 128

In [9]:
layer_conv1, weights_conv1 = \
    createConvLayer(input=x_image,
                   num_input_channels=num_channels,
                   filter_size=filter_size1,
                   num_filters=num_filters1)

In [10]:
layer_conv2, weights_conv2 = \
    createConvLayer(input=layer_conv1,
                   num_input_channels=num_filters1,
                   filter_size=filter_size2,
                   num_filters=num_filters2)

In [11]:
layer_2_shape = layer_conv2.shape

In [12]:
num_features = layer_2_shape[1:4].num_elements()

In [13]:
flat_conv_layer =  tf.reshape(layer_conv2, [-1, num_features])

In [14]:
def new_weights(s):
    return tf.Variable(tf.truncated_normal(s, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [15]:
def new_fc_layer(input,          # The previous layer.
                 num_inputs,     # Num. inputs from prev. layer.
                 num_outputs,
                 use_relu=True): 
    shape=[num_inputs, num_outputs]
    
    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    biases = new_biases(length=num_outputs)

   
    layer = tf.matmul(input, weights) + biases    
    # Use ReLU?
    if use_relu:
        layer = tf.nn.relu(layer)

    return layer

In [16]:
layer_fc1 = new_fc_layer(input=flat_conv_layer,
                         num_inputs=num_features,
                         num_outputs=fc_size,
                         use_relu=True)

In [17]:
layer_fc2 = new_fc_layer(input=layer_fc1,
                         num_inputs=fc_size,
                         num_outputs=class_size,
                         use_relu=False)

In [18]:
y_pred = tf.nn.softmax(layer_fc2)
y_pred_cls = tf.argmax(y_pred, axis=1)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                        labels=y_true)
cost = tf.reduce_mean(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [19]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [20]:
session = tf.Session()
session.run(tf.global_variables_initializer())

In [21]:
batch_size = 64
num_iterations = 8000
data.test.cls = np.array([label.argmax() for label in data.test.labels])

feed_dict_test = {x: data.test.images,
                  y_true: data.test.labels,
                  y_true_cls: data.test.cls}
accuracy_points = []
cost_points = []

In [22]:
for i in range(num_iterations):
        x_batch, y_true_batch = data.train.next_batch(batch_size)
        
        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        if i % 100 == 0:
            acc = session.run(accuracy, feed_dict=feed_dict_test)
            cost_points += [float(cost.eval(feed_dict_train, session))]
            accuracy_points += [float(acc)]
            
            if i % 100 == 0:
                acc = session.run(accuracy, feed_dict=feed_dict_test)
                print("Iteration ", i, " accuracy = ", acc)
        

        session.run(optimizer, feed_dict=feed_dict_train)

Iteration  0  accuracy =  0.0909
Iteration  100  accuracy =  0.6458
Iteration  200  accuracy =  0.8191
Iteration  300  accuracy =  0.8715
Iteration  400  accuracy =  0.8922
Iteration  500  accuracy =  0.9048
Iteration  600  accuracy =  0.9131
Iteration  700  accuracy =  0.9175
Iteration  800  accuracy =  0.9253
Iteration  900  accuracy =  0.9313
Iteration  1000  accuracy =  0.9323
Iteration  1100  accuracy =  0.9372
Iteration  1200  accuracy =  0.9408
Iteration  1300  accuracy =  0.9438
Iteration  1400  accuracy =  0.9436
Iteration  1500  accuracy =  0.9502
Iteration  1600  accuracy =  0.9509
Iteration  1700  accuracy =  0.9534
Iteration  1800  accuracy =  0.9561
Iteration  1900  accuracy =  0.9575
Iteration  2000  accuracy =  0.958
Iteration  2100  accuracy =  0.9619
Iteration  2200  accuracy =  0.9643
Iteration  2300  accuracy =  0.9636
Iteration  2400  accuracy =  0.9637
Iteration  2500  accuracy =  0.968
Iteration  2600  accuracy =  0.9697
Iteration  2700  accuracy =  0.9708
Iterat